In [1]:
import cv2
import shutil
import numpy as np
from pathlib import Path
from ultralytics import YOLO

# ==============================================================================
# 1. CẤU HÌNH HỆ THỐNG
# ==============================================================================
BASE_DIR = Path(r"D:\THS\06.1 Python\AI Camera")

PATHS = {
    "check":      BASE_DIR / "1_Check",
    "helmet":     BASE_DIR / "2_Detect_Helmet",
    "no_helmet":  BASE_DIR / "3_Detect_No_Helmet",
    "no_detect":  BASE_DIR / "5_No Detect",
    "archive":    BASE_DIR / "9_Archieve"
}

# Từ khóa phân loại
KEYWORDS_SAFE = ['helmet', 'hardhat', 'safety-cap']
KEYWORDS_UNSAFE = ['no-helmet', 'head', 'face', 'person', 'no_helmet']

# ==============================================================================
# 2. CHƯƠNG TRÌNH CHÍNH (VẼ RIÊNG TỪNG OBJECT - KHỬ TRÙNG LẶP)
# ==============================================================================
model_path = BASE_DIR / "best.pt"
if not model_path.exists():
    print("❌ Không tìm thấy model best.pt!")
    exit()

model = YOLO(str(model_path))

image_files = list(PATHS["check"].glob('*.jpg')) + list(PATHS["check"].glob('*.png'))

if not image_files:
    print(f"⚠️ Thư mục {PATHS['check']} đang trống.")
else:
    print(f"🚀 Đang xử lý {len(image_files)} ảnh (Vẽ khung riêng biệt cho từng người)...")

    # Sử dụng iou=0.3 để YOLO tự động khử các khung chồng lấn cực mạnh
    # Điều này giúp 1 người chỉ có 1 khung dù conf thấp.
    results = model.predict(source=image_files, conf=0.1, iou=0.3, verbose=False)

    for result in results:
        origin_file = Path(result.path)
        filename = origin_file.name
        img = cv2.imread(str(origin_file))
        
        target_folder = PATHS["no_detect"]
        has_violation = False
        has_safe = False
        
        boxes = result.boxes
        if len(boxes) > 0:
            for box in boxes:
                cls_name = model.names[int(box.cls[0])].lower().replace("_", "-")
                conf_score = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # A. Vẽ khung cho nhóm VI PHẠM (Đỏ)
                is_unsafe = any(kw in cls_name for kw in KEYWORDS_UNSAFE)
                if is_unsafe and conf_score >= 0.1:
                    has_violation = True
                    # Vẽ khung đỏ cho từng người
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    cv2.putText(img, f"NO-HELMET {conf_score:.2f}", (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
                # B. Vẽ khung cho nhóm AN TOÀN (Xanh)
                is_safe = any(kw in cls_name for kw in KEYWORDS_SAFE)
                if is_safe and conf_score >= 0.4:
                    # Chỉ vẽ xanh nếu vùng này chưa bị đánh dấu là đỏ (ưu tiên cảnh báo)
                    has_safe = True
                    if not has_violation:
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(img, f"Helmet {conf_score:.2f}", (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

            # Quyết định thư mục lưu
            if has_violation:
                target_folder = PATHS["no_helmet"]
            elif has_safe:
                target_folder = PATHS["helmet"]

        # Lưu và di chuyển file
        cv2.imwrite(str(target_folder / filename), img)
        shutil.move(str(origin_file), str(PATHS["archive"] / filename))

        status = "❌ VI PHẠM" if has_violation else "✅ AN TOÀN"
        print(f"{status} | {filename}")

print("\n🎉 Xong! Mỗi đối tượng vi phạm đã có khung riêng và không còn bị chồng lấn.")

🚀 Đang xử lý 1 ảnh (Vẽ khung riêng biệt cho từng người)...
❌ VI PHẠM | Pic Check 1.jpg

🎉 Xong! Mỗi đối tượng vi phạm đã có khung riêng và không còn bị chồng lấn.


In [ ]:
import cv2
import shutil
import sys
from pathlib import Path
from ultralytics import YOLO

# ==============================================================================
# 1. CẤU HÌNH & KHỞI TẠO
# ==============================================================================
def setup_system(base_path_str):
    """
    Thiết lập các đường dẫn và tạo thư mục nếu chưa tồn tại.
    Trả về dictionary chứa các đường dẫn.
    """
    base_dir = Path(base_path_str)
    
    paths = {
        "check":      base_dir / "1_Check",
        "helmet":     base_dir / "2_Detect_Helmet",
        "no_helmet":  base_dir / "3_Detect_No_Helmet",
        "no_detect":  base_dir / "5_No Detect",
        "archive":    base_dir / "9_Archieve"
    }

    # Tạo thư mục
    for p in paths.values():
        p.mkdir(parents=True, exist_ok=True)
        
    return base_dir, paths

# ==============================================================================
# 2. HÀM XỬ LÝ CHÍNH
# ==============================================================================
def run_helmet_detection(base_path_str):
    # --- A. Thiết lập ---
    BASE_DIR, PATHS = setup_system(base_path_str)
    
    # Từ khóa phân loại
    KEYWORDS_SAFE = ['helmet', 'hardhat', 'safety-cap']
    KEYWORDS_UNSAFE = ['no-helmet', 'head', 'face', 'person', 'no_helmet']

    # --- B. Load Model ---
    model_path = BASE_DIR / "best.pt"
    if not model_path.exists():
        print(f"❌ Lỗi: Không tìm thấy model tại {model_path}")
        return # Dừng hàm nếu không có model

    print(f"⏳ Đang tải model từ {model_path}...")
    try:
        model = YOLO(str(model_path))
    except Exception as e:
        print(f"❌ Lỗi khi tải model: {e}")
        return

    # --- C. Tìm ảnh đầu vào ---
    image_files = list(PATHS["check"].glob('*.jpg')) + list(PATHS["check"].glob('*.png'))

    if not image_files:
        print(f"⚠️ Thư mục {PATHS['check']} đang trống. Không có ảnh để xử lý.")
        return

    print(f"🚀 Đang xử lý {len(image_files)} ảnh (Chế độ vẽ khung riêng biệt)...")

    # --- D. Chạy nhận diện (Batch Inference) ---
    # Sử dụng iou=0.3 để khử khung chồng lấn, conf=0.1 để bắt lỗi nhạy
    results = model.predict(source=image_files, conf=0.1, iou=0.3, verbose=False)

    # --- E. Xử lý từng kết quả ---
    for result in results:
        origin_file = Path(result.path)
        filename = origin_file.name
        
        # Đọc ảnh gốc
        img = cv2.imread(str(origin_file))
        if img is None:
            print(f"❌ Lỗi đọc file: {filename}")
            continue

        target_folder = PATHS["no_detect"]
        has_violation = False
        has_safe = False
        
        boxes = result.boxes
        if len(boxes) > 0:
            for box in boxes:
                cls_name = model.names[int(box.cls[0])].lower().replace("_", "-")
                conf_score = float(box.conf[0])
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                
                # --- LOGIC VẼ KHUNG ---
                
                # 1. Kiểm tra nhóm VI PHẠM (Ưu tiên vẽ Đỏ)
                is_unsafe = any(kw in cls_name for kw in KEYWORDS_UNSAFE)
                if is_unsafe and conf_score >= 0.1:
                    has_violation = True
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 3)
                    cv2.putText(img, f"NO-HELMET {conf_score:.2f}", (x1, y1 - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                
                # 2. Kiểm tra nhóm AN TOÀN (Vẽ Xanh)
                is_safe = any(kw in cls_name for kw in KEYWORDS_SAFE)
                if is_safe and conf_score >= 0.4:
                    # Chỉ vẽ xanh nếu chưa bị đánh dấu là đỏ (để tránh rối mắt)
                    if not has_violation:
                        has_safe = True
                        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(img, f"Helmet {conf_score:.2f}", (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                    else:
                        # Nếu ảnh đã có vi phạm, vẫn đánh dấu logic safe=True để biết, 
                        # nhưng không vẽ lên hình để tập trung vào lỗi.
                        has_safe = True

            # --- Quyết định thư mục lưu ---
            if has_violation:
                target_folder = PATHS["no_helmet"]
            elif has_safe:
                target_folder = PATHS["helmet"]

        # --- F. Lưu và Di chuyển ---
        # 1. Lưu ảnh kết quả
        output_path = target_folder / filename
        cv2.imwrite(str(output_path), img)
        
        # 2. Di chuyển ảnh gốc sang Archive
        archive_path = PATHS["archive"] / filename
        shutil.move(str(origin_file), str(archive_path))

        status = "❌ VI PHẠM" if has_violation else ("✅ AN TOÀN" if has_safe else "⚪ KHÔNG DETECT")
        print(f"{status} | {filename} -> {target_folder.name}")

    print("\n🎉 Hoàn tất! Đã xử lý xong toàn bộ ảnh.")

# ==============================================================================
# 3. CHẠY CHƯƠNG TRÌNH
# ==============================================================================
if __name__ == "__main__":
    # Bạn chỉ cần đổi đường dẫn ở đây
    working_path = r"D:\THS\06.1 Python\AI Camera"
    
    run_helmet_detection(working_path)